# Step 1 (Enviroment Setup)


---


## Importing the Datasets
wikitext (*full* ) - 859955 docs

wikitext (*small* ) - 10000 docs

In [1]:
!wget -O wikitext-filtered-full.zip "https://www.dropbox.com/scl/fi/ibd4cmixckghx6hhb361c/wikitext-filtered-full.zip?rlkey=q71cebf0k5fvvwhmcntoswzhq&dl=1"
!wget -O wikitext-filtered-10k.zip "https://www.dropbox.com/scl/fi/ek174r3sg7qjx0aa9atop/wikitext-filtered-10k.zip?rlkey=zy6jqxv6qsc16lr9qm3ki9uhf&dl=1"

--2025-10-17 23:04:24--  https://www.dropbox.com/scl/fi/ibd4cmixckghx6hhb361c/wikitext-filtered-full.zip?rlkey=q71cebf0k5fvvwhmcntoswzhq&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uceea71edf314d203d1ecea54905.dl-eu.dropboxusercontent.com/cd/0/inline/CzaM-GcUFl25wnxZLuuZEjPle2s8cbOBXGUdauuTx4bVyCkbHbKJUfztLG3QrW5FRDVIUwJSHM88MN9fg7ti5opxoNh6rR4SszpW_0stmIH7BuoPSPh_MbPtg1K0JHjBqlKuDbSk_AEyCcpoIy8yHpzC/file?dl=1# [following]
--2025-10-17 23:04:25--  https://uceea71edf314d203d1ecea54905.dl-eu.dropboxusercontent.com/cd/0/inline/CzaM-GcUFl25wnxZLuuZEjPle2s8cbOBXGUdauuTx4bVyCkbHbKJUfztLG3QrW5FRDVIUwJSHM88MN9fg7ti5opxoNh6rR4SszpW_0stmIH7BuoPSPh_MbPtg1K0JHjBqlKuDbSk_AEyCcpoIy8yHpzC/file?dl=1
Resolving uceea71edf314d203d1ecea54905.dl-eu.dropboxusercontent.com (uceea71edf314d203d1ecea54905.dl-eu.dropbo

In [4]:
!unzip wikitext-filtered-full.zip
!unzip wikitext-filtered-10k.zip

Archive:  wikitext-filtered-full.zip
replace wikitext-filtered-full/dataset_info.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  wikitext-filtered-10k.zip
replace wikitext-filtered-10k/dataset_info.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [5]:
# datasets package provides dataset tools from hugginface
!pip install datasets
import datasets

In [6]:
from datasets import load_dataset, Dataset

def load_dataset():
  wikitext_small = "wikitext-filtered-10k"
  wikitext_large = "wikitext-filtered-full"

  dataset_small = Dataset.load_from_disk(wikitext_small)
  dataset_large = Dataset.load_from_disk(wikitext_large)
  print("wikitext_small: {} docs, wikitext_large: {} docs".format(len(dataset_small), len(dataset_large)))
  return dataset_small, dataset_large

wikitext_small, wikitext_large = load_dataset()

wikitext_small: 10000 docs, wikitext_large: 859955 docs


## Understanding the Dataset
Summary statistics

In [7]:
wt = wikitext_small
#wt = wikitext_large

print('# TYPE OF THE DATASET:', '\n', type(wt))
print(wt, '\n')
print('# ENTRIES LOOK LIKE:')
print(wt.features, '\n', wt[0], '\n', wt[1], '\n')

print('# DATASET STATISTICS:')
print('No. of docs:', len(wt))
lengths = [len(doc['text'].split()) for doc in wt]
print('Mean doc length:', sum(lengths)/len(lengths), 'words')

# TYPE OF THE DATASET: 
 <class 'datasets.arrow_dataset.Dataset'>
Dataset({
    features: ['text'],
    num_rows: 10000
}) 

# ENTRIES LOOK LIKE:
{'text': Value('string')} 
 {'text': 'Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " .'} 
 {'text': "The game began development in 2010 , carrying over a large portion of the work done on Valkyria Ch

# Step 2 (Train Baselines)

---
Installing dependancies
- gensim - word2vec models
- nltk (natural language tool kit) - stopwords removal

In [8]:
!pip install gensim nltk
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

In [9]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess_dataset(dataset):
    text_col = 'text' if 'text' in dataset.column_names else dataset.column_names[0]
    tokenized = []

    for i in range(len(dataset)):
        text = dataset[i][text_col]
        if not isinstance(text, str):
            continue
        tokens = [t.lower() for t in text.split() if t.isalpha() and t.lower() not in stop_words]
        if tokens:
            tokenized.append(tokens)

    return tokenized

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
tokens_small = preprocess_dataset(wikitext_small)
tokens_large = preprocess_dataset(wikitext_large)

In [11]:
def train_word2vec(tokens, model_name, vector_size=50, window=5, min_count=5, epochs=5):
    print(f"Training {model_name} ...")
    model = Word2Vec(
        sentences=tokens,
        vector_size=vector_size,
        window=window,
        min_count=min_count,
        epochs=epochs
    )
    model.save(f"{model_name}.model")
    model.wv.save(f"{model_name}.kv")
    print(f"{model_name} saved.")
    return model

In [12]:
model_small = train_word2vec(tokens_small, "word2vec_small")
model_large = train_word2vec(tokens_large, "word2vec_large")

Training word2vec_small ...
word2vec_small saved.
Training word2vec_large ...
word2vec_large saved.


In [ ]:
# writing model to disk
model_small.save("word2vec_small.model")
model_large.save("word2vec_large.model")

In [ ]:
model_small = Word2Vec.load("word2vec_small.model")
model_large = Word2Vec.load("word2vec_large.model")

In [ ]:
vocab_size = len(model_small.wv)
print(f"Vocab size (learned by model):", vocab_size, '\n')

example_tokens = ['plane', 'car', 'planet', 'nurse', 'city', 'country']
for token in example_tokens:
    if token in model_small.wv:
        print(f"Top-10 similar to '{token}':", model_small.wv.most_similar(token, topn=10))
    else:
        print(f"'{token}' not in vocabulary.")

Vocab size (learned by model): 162898 

Top-10 similar to 'plane': [('airplane', 0.7798449397087097), ('planes', 0.7587494850158691), ('helicopter', 0.7551898956298828), ('flight', 0.7514647245407104), ('wreckage', 0.7437148690223694), ('airliner', 0.7287061214447021), ('takeoff', 0.7201843857765198), ('saucer', 0.7154824733734131), ('cockpit', 0.7090662717819214), ('amana', 0.701010525226593)]
Top-10 similar to 'car': [('cars', 0.8536964058876038), ('truck', 0.8398393988609314), ('motorcycle', 0.8149896860122681), ('lorry', 0.801920473575592), ('suv', 0.7947259545326233), ('vehicle', 0.7883414030075073), ('driver', 0.7828571200370789), ('cab', 0.7772312164306641), ('taxi', 0.7629276514053345), ('tires', 0.7254185080528259)]
Top-10 similar to 'planet': [('earth', 0.8504156470298767), ('galaxy', 0.835283100605011), ('pluto', 0.7836962938308716), ('planets', 0.7812984585762024), ('asteroid', 0.7718139886856079), ('uranus', 0.7695909738540649), ('universe', 0.7661747932434082), ('jupiter'

# Task 3

In [ ]:
import numpy as np
import math

In [ ]:
def mag(v):
  s = sum((e*e) for e in v)
  s = math.sqrt(s)
  return s

def cosineSimilarity(v1, v2):
  dotProd = np.dot(v1, v2)
  cos = dotProd/(mag(v1)*mag(v2))
  return cos

In [ ]:
planeIndex = model_small.wv.key_to_index['plane']
carIndex = model_small.wv.key_to_index['car']
planetIndex = model_small.wv.key_to_index['planet']
sunIndex = model_small.wv.key_to_index['sun']
passengerIndex = model_small.wv.key_to_index['passenger']

In [ ]:
v1 = model_large.wv['planet']
v2 = model_large.wv['sun']

cosineSimilarity(v1, v2)

0.6306870286899026

# Step 4

In [23]:
!wget -q -O combined.csv "https://raw.githubusercontent.com/kavgan/nlp-text-mining-working-examples/master/wordSimilarity/resources/wordsim353/combined.csv"


In [14]:
!pip -q install pandas scipy
import pandas as pd
import numpy as np
from scipy.stats import spearmanr

In [24]:
!wget -q -O combined.zip "https://raw.githubusercontent.com/kavgan/nlp-text-mining-working-examples/master/wordSimilarity/resources/wordsim353/combined.csv"

!unzip -o combined.zip

import pandas as pd

df = pd.read_csv("combined.csv")  # columns: 'Word 1', 'Word 2', 'Human (mean)'
print("Loaded: wordsim353/combined.csv | shape:", df.shape)
print(df.head())

Archive:  combined.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of combined.zip or
        combined.zip.zip, and cannot find combined.zip.ZIP, period.


EmptyDataError: No columns to parse from file